In [156]:
import numpy as np
import openpyxl
from pandas import DataFrame, read_csv, merge
import matplotlib.pyplot as plt
%matplotlib inline

In [147]:
loan_df = read_csv("loan.csv")

loan_df["loan_type"] = [val.lower().strip() if isinstance(val, str) else val 
                        for val in loan_df.loan_type ]

In [148]:
# get approved loans

appr = loan_df.loc[loan_df.status == 'approved']
appr.index = range(appr.shape[0])

# get Liberian dollar loans

ld_loan = DataFrame( appr.loc[~appr.loan_amt_ld.isnull()] )
ld_loan["loan_amt_ld"] = [float(num) for num in ld_loan["loan_amt_ld"]]
ld_loan.index = range(ld_loan.shape[0])

# get US dollar loans

usd_loan = DataFrame( appr.loc[~appr.loan_amt_usd.isnull()] )
usd_loan["loan_amt_usd"] = [float(num) for num in usd_loan["loan_amt_usd"]]
usd_loan.index = range(usd_loan.shape[0])

In [149]:
# get stats by loan_type

ld_ln_grp = ld_loan.groupby("loan_type", as_index=False).agg({"loan_id": "count", 
                                                              "loan_amt_ld": "sum"})
ld_ln_grp = ld_ln_grp.rename(columns = {"loan_id": "num_ld_loans"})

usd_ln_grp = usd_loan.groupby("loan_type", as_index=False).agg({"loan_id": "count", 
                                                               "loan_amt_usd": "sum"})
usd_ln_grp = usd_ln_grp.rename(columns = {"loan_id": "num_usd_loans"})

# merge groups

merged_ln_grps = merge(ld_ln_grp, usd_ln_grp, on="loan_type", how="outer")

# add total row

row = merged_ln_grps.values.sum(axis=0)
row[0] = "total"

merged_ln_grps.loc[-1] = row
merged_ln_grps.index = range(merged_ln_grps.shape[0])

In [150]:
merged_ln_grps

,loan_type,num_ld_loans,loan_amt_ld,num_usd_loans,loan_amt_usd
0,business,7,138000.0,2,400.0
1,personal,2,17250.0,4,500.0
2,total,9,155250.0,6,900.0


In [151]:
5000 - ( (155250.0 / 115 ) + 900.0)

2750.0

In [152]:
merged_ln_grps.to_csv("weekly_loan_summary.csv", header=True, index=False)

In [159]:
merged_ln_grps.to_excel("weekly_loan_summary.xlsx", index=False)

In [181]:
import smtplib
# import email.mime as mime

from email.mime.message import MIMEMessage, message
# Import the email modules we'll need
from email.mime.text import MIMEText
